# The Homology Module

This chapter is devoted to the description of the functions for computing the homology groups of a chain complex.

## List of functions

``(chcm-homology chcm dim)`` *[Function]*
> Return a description of the homology group in dimension *dim* of the chain complex *chcm* in terms of *components* of the form $\mathbb{Z}$ or $\mathbb{Z}/{n\, \mathbb{Z}}$. The desired homology group is the direct sum of these  components. No canonical presentation is looked for, so that, if for instance, a homology group is $\mathbb{Z}/{6\, \mathbb{Z}}$, it can be displayed as one component
$\mathbb{Z}/{6\, \mathbb{Z}}$ or two components $\mathbb{Z}/{2\, \mathbb{Z}}$ and $\mathbb{Z}/{3\, \mathbb{Z}}$. On the other hand, if the component part  is void, this  means  the homology group is the null group. The function ``chcm-homology`` implements the usual algorithms to compute the homology group associated to two integer matrices, the composite of which is null. The current version is verbose: for each group asked for, the program displays the rank of each integer matrix and each generator of the source module. Timing indications are also given. In the examples, only the  components are printed.

``{chcm-homology-gen chcm n)`` *[Function]*
> This function computes the homology group in dimension $n$ of the chain complex *chcm* and prints a generator of degree $n$ in *chcm* for each component of the group (in general a combination of the  basis elements of degree $n$ of the chain complex.

``(chcm-mat chcm n)`` *[Function]*
> Return the matrix of the linear homomorphism  $d_n: C_n \rightarrow C_{n-1}$, where $C_n$ is the $n$--th chain group of the chain complex *chcm*. More precisely, each column  of this matrix contains the integer coefficients in the basis of $C_{n-1}$ of the image by $d_n$ of each basis element of $C_n$. So the number of lines (resp. columns) of the matrix is the number of elements of the basis of $C_{n-1}$ (resp. $C_n$). The homology group ${\cal H}_n={\cal Z}_n/{\cal B}_n$ is computed from the two matrices ``MZ`` $=$ ``chcm-mat`` *(chcm,n)* and ``NB`` $=$ ``chcm-mat`` *(chcm, n+1)*. By well known algorithms on matrix reduction (**S. MacLane & G. Birkhoff**, *Algebra*, The MacMillan Company, 1967), the matrix ``MZ`` is used to find a basis for the kernel ${\cal Z}_n$ and the matrix ``NB``, to find in ${\cal Z}_n$ a presentation of the group ${\cal H}_n={\cal Z}_n/{\cal B}_n$ by generators and relations. This is performed by the internal function ``homologie`` (beware: not ``homology``!).

``(homology chcm  degr1 &optional (degr2 (1+ degr1)))`` *[Method]*
> Compute the homology groups from $degr_1$ to $degr_2-1$ (default: only $degr_1$, if *degr2* is omitted) of the right chain complex of the homotopy equivalence contained in the slot ``:efhm`` of the
chain complex instance {\em chcm}. At the creation of the chain complex, this slot is unbound and is set during execution by an adequate CLOS method. If this slot cannot be  bound, an error is returned and the homology groups cannot be computed. A more elaborated explanation of the mechanism used by the function ``homology`` is given in the  section: **The general method for computing homology groups.**


## Example

Let us get the homology groups for the example ``diabolo`` of the chapter 1.

In [ ]:
(def diabolo (build-chcm
              :cmpr #'s-cmpr
              :basis #'(lambda (dmn)
                               (case dmn
                                   (0 '(s0 s1 s2 s3 s4 s5))
                                   (1 '(s01 s02 s12 s23 s34 s35 s45))
                                   (2 '(s345))
                                   (otherwise nil)))
              :bsgn 's0
              :intr-dffr #'(lambda (dmn gnr)
                                   (unless (<= 0 dmn 2)
                                       (error "Incorrect dimension for diabolo-dp."))
                                   (case dmn
                                       (0 (cmbn -1))  ; Note the null combination of degree -1
                                       (1 (case gnr
                                              (s01 (cmbn 0 -1 's0 1 's1))
                                              (s02 (cmbn 0 -1 's0 1 's2))
                                              (s12 (cmbn 0 -1 's1 1 's2))
                                              (s23 (cmbn 0 -1 's2 1 's3))
                                              (s34 (cmbn 0 -1 's3 1 's4))
                                              (s35 (cmbn 0 -1 's3 1 's5))
                                              (s45 (cmbn 0 -1 's4 1 's5))))
                                        (2 (case gnr
                                               (s345 (cmbn 1 1 's34 -1 's35 1 's45))))
                                       (otherwise (error "Bad generator for complex diabolo"))))
              :strt :GNRT
              :orgn '(diabolo-for-example)))

In [ ]:
(chcm-homology diabolo 0)

In [ ]:
(chcm-homology diabolo 1)

In [ ]:
(chcm-homology diabolo 2)

Another simple example is the following  2-chain  complex  corresponding to the well known *dunce hat*. We shall see later in the chapter ``Simplicial Sets`` a much more elegant method to describe this object.

In [ ]:
(cl-jupyter-user:png-from-file "dunce.png")

The diagram shows a permissible triangulation. For the generators, we have  chosen lists rather symbols: a vertex $s_i$ is represented as ``(i)``, an edge $s_is_j$ as ``(i j)`` and a triangle $s_is_js_k$ as ``(i j k)``. The enumeration of the elements of the basis is a little  cumbersome but defining  the boundary homomorphism is very easy, bearing in mind the boundary rule:

$$
\mathbf{d}[s_0s_1\ldots s_n] = \sum_{i=0}^n{(-1)^is_0s_1\ldots\widehat{s_i}\ldots s_n}.
$$

As for diabolo, the vertices are implicitly ordered.

In [ ]:
(def duncehat-basis #'(lambda (dmn)
                              (case dmn
                                  (0 '((0) (1) (2) (3) (4) (5) (6) (7)))
                                  (1 '((0 1) (0 2) (0 3)
                                       (0 4) (0 5) (0 6)
                                       (0 7) (1 2) (1 3)
                                       (1 4) (1 5) (1 6)
                                       (1 7) (2 3) (2 4)
                                       (2 6) (2 7) (3 4)
                                       (3 5) (4 5) (4 6)
                                       (5 6) (5 7) (6 7)))
                                  (2 '((0 1 5) (0 1 6) (0 1 7)
                                       (0 2 3) (0 2 4) (0 2 6)
                                       (0 3 4) (0 5 7) (1 2 3)
                                       (1 2 4) (1 2 7) (1 3 5)
                                       (1 4 6) (2 6 7) (3 4 5)
                                       (4 5 6) (5 6 7)))
                                  (otherwise nil))))

In [ ]:
(def duncehat-df #'(lambda (dmn gnr)
                           (case dmn
                               (0 (cmbn -1))
                               (1 (cmbn 0 -1 (list (first gnr)) 1 (rest gnr)))
                               (2 (cat:cmbn 1 1 (list (first gnr) (second gnr))
                                             -1 (list (first gnr) (third gnr))
                                              1 (rest gnr)))
                               (otherwise nil))))

In [ ]:
(def duncehat (build-chcm :cmpr #'l-cmpr
                          :basis duncehat-basis
                          :bsgn '(0)
                          :intr-dffr duncehat-df
                          :strt :gnrt
                          :orgn '(dunce hat)))

In [ ]:
(chcm-homology duncehat 0)

In [ ]:
(chcm-homology duncehat 1)

In [ ]:
(chcm-homology duncehat 2)

Let us take again  the chain complex ``duncehat``. The two matrices of the homomorphisms $d_1: C_1 \rightarrow C_0$ and $d_2: C_2 \rightarrow C_1$ are obtained by calling ``chcm-mat``.

In [ ]:
(def mz (chcm-mat duncehat 1))

In [ ]:
(def nb (chcm-mat duncehat 2))

In [ ]:
(homologie mz nb)

## The general method for computing homology

Among the slots of the instance of an object inheriting the ``CHAIN COMPLEX`` class, a slot ``efhm``
has been reserved to point (possibly) to a homotopy equivalence where the right bottom chain complex is effective. The function ``homology`` is designed to get the right bottom chain complex of the homotopy equivalence value of the slot. If the slot has been bound, then the homology groups are computed by the function ``chcm-homology`` as shown in the following definition:

```lisp
(DEFMETHOD HOMOLOGY ((chcm chain-complex) degr1 &optional (degr2 (1+ degr1)))
   (do ((degr degr1 (1+ degr)))
       ((>= degr degr2))
 -->   (chcm-homology (rbcc (efhm chcm)) degr)
       (terpri) (clock) (terpri)))
```

But, at the creation of the object, the slot ``efhm`` is  unbound and as soon as the ``homology``
function tries to get the content of the slot ``efhm`` via the call ``(efchm chcm)``, the *slot-unbound* mechanism of CLOS is triggered, calling at its turn a method ``search-efhm`` depending on the object. If no method is available for this object, ``NIL`` is returned. In the following chapters, we shall see that cases have been written for cartesian products, tensor products, suspensions, disk pasting, fibrations, loop spaces, and classifying spaces. The selection of the case is done thanks to the information contained in the comment list of the object (slot ``orgn``). In each case, the ``search-efhm`` method builds - in general with a complex machinery, including a possible recursivity - a homotopy equivalence where the right bottom chain complex is *effective*. The slot ``efhm`` of the object is then settled and the homology group computation may begin. If ``NIL`` is returned, then  and only at this moment, the slot-unbound mechanism looks if the chain complex is finite (checking if a basis function exists). If this is the case, then the *trivial homotopy equivalence* is built upon the chain complex and this gives the value of the slot ``efhm``. If there is no basis function,  meaning that probably the chain complex is locally effective, an error is returned.

*Remark.* The user may wonder why one does not  look first if the given object is effective. We recall simply that even in the case of an effective chain complex, it is sometimes possible to find another effective chain complex, homotopic to the first and whose number of basis elements, in any dimensions, is considerably smaller in comparison with the first one. A striking case will be shown in a further chapter, showing the application of the Eilenberg-Zilber theorem to a cartesian product.


### Example

*The reading of this subsection may be postponed until a full reading of this user's guide.* The following examples  show how are filled the slots ``efhm`` of the various objects. Starting from the sphere $S^2$, we  verify first that the *efhm* slot is unbound. Then we ask for ${\cal H}_1$ and verify that, though there is a finite basis, ``Kenzo`` has nevertheless built a trivial homotopy equivalence on this object.

In [ ]:
(cat-init)

In [ ]:
(def s2 (cat:sphere 2))

In [ ]:
(inspect s2)

In [ ]:
(homology s2 1)

In [ ]:
(inspect s2)

In [ ]:
(orgn (cat:hmeq 9))

Now, we create $\Omega^1(S^2)$. Getting  the value of the slot ``efhm`` by a call to the accessor function ``efhm``, triggers the search-efhm method for a loop space. A homotopy equivalence is built and the slot is set.

In [ ]:
(def os2 (loop-space s2))

In [ ]:
(inspect os2)

In [ ]:
(orgn os2)

In [ ]:
(efhm os2)

In [ ]:
(inspect os2)

The following example shows the recursion mechanism when one wants to get the value of the slot ``efhm`` of an iterated loop space, namely $\Omega^3(S^4)$.

In [ ]:
(def s4 (sphere 4))

In [ ]:
(inspect s4)

In [ ]:
(def ooos4 (loop-space (loop-space (loop-space s4))))

In [ ]:
(orgn ooos4)

In [ ]:
(orgn (second *))

In [ ]:
(orgn (second *))

In [ ]:
(orgn (second *))

In [ ]:
(inspect (smgr 136))

In [ ]:
(inspect (smgr 124))

In [ ]:
(inspect (smst 119))

In [ ]:
(efhm ooos4)

In [ ]:
(inspect (smgr 124))

In [ ]:
(inspect (smst 119))

In [ ]:
(inspect s4)

#### Lisp files concerned in this chapter}

``homology-groups.lisp``, ``searching-homology`` and files containing a ``search-efhm`` method.